### Extraindo os discursos dos parlamentares
Utilizando Web Scraping para obter os discursos dos parlamentares

In [1]:
import bs4
import urllib.request as urllib_request
import pandas as pd

print("BeautifulSoup ->", bs4.__version__)
print("urllib ->", urllib_request.__version__)
print("pandas ->", pd.__version__)

BeautifulSoup -> 4.7.1
urllib -> 3.7
pandas -> 0.24.2


In [2]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError

# função que faz tratamento do html (exclui strings que podem dar erro na extração de dados)
def trata_html(input):
    return " ".join(input.split()).replace('> <', '><')

def extraihtml(url, htmldecode):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}
    try:
        req = Request(url, headers = headers)
        response = urlopen(req)
        html = response.read()
#        html = html.decode('utf-8')
#        html = html.decode('cp1252')
        html = html.decode(htmldecode)
        html = trata_html(html)
    #    print(html)
        soup = BeautifulSoup(html, 'html.parser')
        return soup

    except HTTPError as e:
        return (e.status, e.reason)

    except URLError as e:
        return (e.reason)

In [3]:
# alterar a data e quantidade de registros no link via get
# deverá ser alterada o período dos discursos pela url
#url = 'https://www.camara.leg.br/internet/sitaqweb/resultadoPesquisaDiscursos.asp?txOrador=&txPartido=&txUF=&dtInicio=01%2F03%2F2020&dtFim=14%2F03%2F2020&txTexto=&txSumario=&basePesq=plenario&CampoOrdenacao=dtSessao&PageSize=100&TipoOrdenacao=DESC&btnPesq=Pesquisar'
url = 'https://www.camara.leg.br/internet/sitaqweb/resultadoPesquisaDiscursos.asp?txOrador=&txPartido=&txUF=&dtInicio=01%2F12%2F2019&dtFim=31%2F12%2F2019&txTexto=&txSumario=&basePesq=plenario&CampoOrdenacao=dtSessao&PageSize=1500&TipoOrdenacao=DESC&btnPesq=Pesquisar'

html = extraihtml(url, 'utf-8')
#print(html.prettify()) # .preetify() serve para identar o html

In [4]:
# Separando a tabela do código html
table = html.find('table', class_='table table-bordered variasColunas')
#print(table)

In [5]:
#gerando a lista em colunas
A=[]
B=[]
C=[]
D=[]
E=[]
F=[]
G=[]
H=[]

for row in table.findAll("tr"): #para tudo que estiver em <tr>
    cells = row.findAll('td') #variável para encontrar <td>
    if len(cells)==8: #número de colunas
        A.append(cells[0].find(text=True)) #iterando sobre cada linha
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
        D.append(cells[3].find('a').get('href')) #tratando para se obter o link do campo
        E.append(cells[4].find('a').text)
        F.append(cells[5].find(text=True))
        G.append(cells[6].find(text=True))
        H.append(cells[7].find(text=True))

In [6]:
# Trasnformando em um dataframe
df = pd.DataFrame()

df['Data']=A
df['Sessão']=B
df['Fase']=C
df['Discurso']=D
df['Sumário']=E
df['Orador']=F
df['Hora']=G
df['Publicação']=H

df.head()

,Data,Sessão,Fase,Discurso,Sumário,Orador,Hora,Publicação
0,18/12/2019,428.2019,ENCERRAMENTO,TextoHTML.asp?etapa=5 &nuSessao=428.2019 &nuQu...,,"PAES LANDIM, PTB-PI",19h00,DCD19/12/2019 PAG. 107
1,18/12/2019,428.2019,ENCERRAMENTO,TextoHTML.asp?etapa=5 &nuSessao=428.2019 &nuQu...,,"CARMEN ZANOTTO, CIDADANIA-SC",19h00,DCD19/12/2019 PAG. 105
2,18/12/2019,428.2019,ENCERRAMENTO,TextoHTML.asp?etapa=5 &nuSessao=428.2019 &nuQu...,,"CAPITÃO ALBERTO NETO, REPUBLICANOS-AM",19h00,DCD19/12/2019 PAG. 108
3,18/12/2019,428.2019,COMUNICAÇÕES PARLAMENTARES,TextoHTML.asp?etapa=5 &nuSessao=428.2019 &nuQu...,,"POMPEO DE MATTOS, PDT-RS",18h52,DCD19/12/2019 PAG. 102
4,18/12/2019,428.2019,COMUNICAÇÕES PARLAMENTARES,TextoHTML.asp?etapa=5 &nuSessao=428.2019 &nuQu...,,"JÚLIO CESAR, PSD-PI",18h52,DCD19/12/2019 PAG. 103


In [7]:
def isset(nameVar):
    return nameVar in globals()

nome_orador = []
partido = []
estado = []
for orador in df["Orador"]:
    nome_orador.append(orador.split(", ")[0])
    if ', ' in orador:
        if '-' in orador: 
            partido.append(orador.split(", ")[1].split("-")[0])
            try:
                estado.append(orador.split(", ")[1].split("-")[1])
            except:
                estado.append("-")
        else:
            partido.append("-")
            estado.append("-") 
    else:
        partido.append("-")
        estado.append("-")        


df["NomeOrador"] = nome_orador
df["Partido"] = partido
df["Estado"] = estado

df = df.drop(columns='Sumário')
df = df.drop(columns='Orador')
df.head()

,Data,Sessão,Fase,Discurso,Hora,Publicação,NomeOrador,Partido,Estado
0,18/12/2019,428.2019,ENCERRAMENTO,TextoHTML.asp?etapa=5 &nuSessao=428.2019 &nuQu...,19h00,DCD19/12/2019 PAG. 107,PAES LANDIM,PTB,PI
1,18/12/2019,428.2019,ENCERRAMENTO,TextoHTML.asp?etapa=5 &nuSessao=428.2019 &nuQu...,19h00,DCD19/12/2019 PAG. 105,CARMEN ZANOTTO,CIDADANIA,SC
2,18/12/2019,428.2019,ENCERRAMENTO,TextoHTML.asp?etapa=5 &nuSessao=428.2019 &nuQu...,19h00,DCD19/12/2019 PAG. 108,CAPITÃO ALBERTO NETO,REPUBLICANOS,AM
3,18/12/2019,428.2019,COMUNICAÇÕES PARLAMENTARES,TextoHTML.asp?etapa=5 &nuSessao=428.2019 &nuQu...,18h52,DCD19/12/2019 PAG. 102,POMPEO DE MATTOS,PDT,RS
4,18/12/2019,428.2019,COMUNICAÇÕES PARLAMENTARES,TextoHTML.asp?etapa=5 &nuSessao=428.2019 &nuQu...,18h52,DCD19/12/2019 PAG. 103,JÚLIO CESAR,PSD,PI


In [8]:
# início do link 'https://www.camara.leg.br/internet/sitaqweb'

#Exportando para csv
#df.to_csv('dados/linkdiscursos_04_19.csv', sep=';', index=False)

#### Extraindo os discursos

In [9]:
def extraiDiscurso(urlDiscurso):
    try:
        url2 = 'https://www.camara.leg.br/internet/sitaqweb' + urlDiscurso.lower().replace("ç", "c").replace("ã", "a").replace("á", "a").replace("â", "a").replace("à", "a").replace("ê", "e").replace("é", "e").replace("í", "i").replace("ó", "o").replace("õ", "o").replace("ô", "o").replace("ú", "u").replace("ü", "u")
        html2 = extraihtml(url2, 'cp1252')
        #print(html2.prettify()) # .preetify() serve para identar o html
        # Separando o discurso do código html
        texto = html2.find('p').text
        return(texto)
    except:
        return("i n d i s p o n i v e l")

In [10]:
texto_discurso = []
for x in df.Discurso:
    #print(x)
    texto_discurso.append(extraiDiscurso("/"+x.replace(' ','')))

In [11]:
# Adicionando os discursos ao dataframe
df['TextoDiscurso'] = texto_discurso

In [12]:
df.head()

,Data,Sessão,Fase,Discurso,Hora,Publicação,NomeOrador,Partido,Estado,TextoDiscurso
0,18/12/2019,428.2019,ENCERRAMENTO,TextoHTML.asp?etapa=5 &nuSessao=428.2019 &nuQu...,19h00,DCD19/12/2019 PAG. 107,PAES LANDIM,PTB,PI,DISCURSO NA ÍNTEGRA ENCAMINHADO PELO SR. DEPU...
1,18/12/2019,428.2019,ENCERRAMENTO,TextoHTML.asp?etapa=5 &nuSessao=428.2019 &nuQu...,19h00,DCD19/12/2019 PAG. 105,CARMEN ZANOTTO,CIDADANIA,SC,DISCURSO NA ÍNTEGRA ENCAMINHADO PELA SRA. DEP...
2,18/12/2019,428.2019,ENCERRAMENTO,TextoHTML.asp?etapa=5 &nuSessao=428.2019 &nuQu...,19h00,DCD19/12/2019 PAG. 108,CAPITÃO ALBERTO NETO,REPUBLICANOS,AM,DISCURSO NA ÍNTEGRA ENCAMINHADO PELO SR. DEPU...
3,18/12/2019,428.2019,COMUNICAÇÕES PARLAMENTARES,TextoHTML.asp?etapa=5 &nuSessao=428.2019 &nuQu...,18h52,DCD19/12/2019 PAG. 102,POMPEO DE MATTOS,PDT,RS,O SR. POMPEO DE MATTOS (PDT - RS. Pela ordem....
4,18/12/2019,428.2019,COMUNICAÇÕES PARLAMENTARES,TextoHTML.asp?etapa=5 &nuSessao=428.2019 &nuQu...,18h52,DCD19/12/2019 PAG. 103,JÚLIO CESAR,PSD,PI,O SR. JÚLIO CESAR (PSD - PI. Pela ordem. Sem ...


In [13]:
#Exportando para csv
#df.to_csv('discursos/discursos_01_02_2019.csv', sep=';', index=False)